# Implementación del método

Este notebook muestra paso a paso cómo funciona el método de parametrización para el mapeo estándar. Recordemos que antes ya se calcularon los puntos fijos del mapeo y se hizo un análisis de los mismos por lo que nos concentraremos en sólo aplicar de manera clara y paso a paso el método. Hacerlo de esta manera nos ayudará a comprender mejor cómo se implementó método de manera general. 
Dado que conocemos ya los puntos fijos entonces nos concentraremos en calcular la variedad inestable, la cual se podrá elegir en términos de escoger el valor propio con el signo negativo que ya se calculó en el N3. 

En primer lugar hay que notar que como queremos implementar eso de manera general tenemos que hacer el cálculo de los valores propios en un inicio. Luego de esto se puede proceder con el calculo de cada variedad. 


Como ya sabemos cuáles son los puntos fijos necesitamos ahora calcular los polinomios que serán la parametrización de la variedad.Para ello debemos resolver la ecuación de invariancia.
$$F_{\epsilon}\circ P=P\circ \lambda$$

In [57]:
#usaremos el paquete de TaylorSeries
using TaylorSeries
using LinearAlgebra


In [7]:
#necesitamos definir el mapeo estandar 
"""EstandarMap(θ,p,k)
   
   Función que define el mapeo estándar.
  

   Argumentos:
   -θ : posición
   -p : momento
   -k : constante del mapeo
"""
function EstandarMap(θ,p,k)
    
    θ_n = mod2pi(θ+p)
    p_n = mod2pi(p+k*sin(θ_n))
    
    return [θ_n,p_n]
    
    
end
  

EstandarMap

Definimos el valor de la constante del mapeo que usaremos en este caso. 

In [3]:
ke=0.3

0.3

                                                     
                                                    ORDEN 1 


Queremos que $P_{\theta}$, $P_{p}$ sean polinomios, entonces necesitamos construirlos como dos polinomios de los cuales solo sabemos su primer coeficiente.La idea es ir calculando los términos en cada iteración.

Escribirmos los coeficientes que buscamos como un polinomio a su vez de orden mayor a uno.

In [4]:
#Definimos a θ, p como polinomios orden dos, eso no influye en el orden del coeficiente que estamos calculando
θ,p = set_variables(Float64,"θ p",order=2)

2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖³)
  1.0 p + 𝒪(‖x‖³)

In [5]:
#Necesitamos los polinomios que parametrizan a las variedades, estos serán tipo Taylor1.TaylorN
P_θ=Taylor1([θ], 1)
P_p=Taylor1([p], 1)
#el orden de estos ya es importante para cada iteración,
# aquí es uno porque estamos en el coeficiente de primer grado
print(P_θ,P_p)


  1.0 θ + 𝒪(‖x‖³) + 𝒪(t²)  1.0 p + 𝒪(‖x‖³) + 𝒪(t²)

In [6]:
#Aplicamos el mapeo a estos dos polinomios
Orden1=EstandarMap(P_θ,P_p,ke)

2-element Array{Taylor1{TaylorN{Float64}},1}:
   1.0 θ + 1.0 p + 𝒪(‖x‖³) + 𝒪(t²)
   0.3 θ + 1.3 p + 𝒪(‖x‖³) + 𝒪(t²)

Hasta este momento tenemos calculada la parte izquierda de la ecuación cohomológica 
$$\begin{eqnarray}
\mathbf{f} \circ \mathcal{P} =\mathcal{P}  \circ g
\label{Ecua de invariancia}
\end{eqnarray}
$$

La forma más fácil para resolver esta ecuación  es escribirla como un sistema en forma matricial. 
$$\mathbb{A}\textbf{v}=\textbf{w} $$
Donde la matriz $\mathbb{A}$ contiene los coeficientes que aparecen en los polinomios de acuerdo a cada variable. Mientras que $\textbf{v}$ representa el vector $(\theta,p)$ y $\textbf{w}$ el vector de términos independientes de los polinomios. 

Para escribirlo asi usaremos una función dentro de $\texttt{TaylorSeries}$ que es el jacobiano. 
$$ \left( \begin{array}{cc}
\frac{\partial f_{x}}{\partial \theta} & \frac{\partial f_{p}}{\partial \theta}  \\
\frac{\partial f_{x}}{\partial p} & \frac{\partial f_{p}}{\partial p}  \end{array} \right) $$

In [92]:
#al calcular el jacobiano obtendremos sólo los valores de los coeficientes los cuales 
# son de tipo TaylorN
JPO=jacobian(Orden1)


2×2 Transpose{Taylor1{Float64},Array{Taylor1{Float64},2}}:
  1.0 + 𝒪(t²)   1.0 + 𝒪(t²)
  0.3 + 𝒪(t²)   1.3 + 𝒪(t²)

In [96]:
#como los valores que contiene el jacobiano son de tipo TaylorN y para usarlos como necesitamos
# se deben tener en Float64 vamos a obtenerlos de la siguente manera. 

JacFl = Array{Float64,2}(undef, 2, 2)
for ind in 1:4
    JacFl[ind] = JPO[ind][0]
end
#usamos la matriz JacFl como auxiliar para encontrar los valores flotantes
@show(JacFl)

JacFl = [1.0 1.0; 0.3 1.3]


2×2 Array{Float64,2}:
 1.0  1.0
 0.3  1.3

La matriz que aparece arriba es una que ya podemos usar para calcular los valores y vectores propios. Necesitamos sólo uno de ellos.

In [97]:
# eigen() es una función que calcula los valores y vectores propios de una matriz y los da en ese orden en forma de vector.
eigen_val,eigen_vec = eigen(JacFl)


Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
2-element Array{Float64,1}:
 0.5821091654199727
 1.7178908345800274
eigenvectors:
2×2 Array{Float64,2}:
 -0.922675  -0.812346
  0.385578  -0.583176

In [100]:
# dado que usaremos la variedad inestable, entonces usamos el segundo valor propio
λ=eigen_val[1]
print(λ,eigen_vec)



0.5821091654199727[-0.922675 -0.812346; 0.385578 -0.583176]

Tomamos por ende el segundo vector propio, los coeficientes de este vector serán los coeficientes de primer orden en los polinomios. 

In [108]:
vec1=[eigen_vec[3],eigen_vec[4]]
# decimos que vec es un arreglo de contiene los valores correspondientes a los lugares [1,2], [2,2] de eve.

2-element Array{Float64,1}:
 -0.8123460094785507
 -0.5831757547123116

In [109]:
#queremos revisar la estabilidad de tipo de las variables Pθ,\theta
@show(P_θ)
@show(typeof(P_θ))
@show(Orden1[1])
@show(typeof(Orden1[1]))


P_θ =   1.0 θ + 𝒪(‖x‖³) + 𝒪(t²)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
Orden1[1] =   1.0 θ + 1.0 p + 𝒪(‖x‖³) + 𝒪(t²)
typeof(Orden1[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                  ORDEN 2

A partir de este término la forma de obtener los coeficientes se vuelve monótona, esto facilita que podamos sintetizar
todos los pasos en funciones. Antes de eso calcularemos cada término por separado. La forma de comenzar a calcular cada coeficiente no cambia con respecto al primero. 

In [110]:

#Queremos hacer la misma idea para este orden, recordemos que el orden de este polinomio no influye por ahora 
# en el cálculo, ya que en los términos que no aparecen explícitamente sirven 
# para calcular la parte que no conocemos.

θ,p = set_variables(Float64,"θ p",order=1)

2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖²)
  1.0 p + 𝒪(‖x‖²)

Para los nuevos polinomios agregamos los valores que calculamos en el paso anterior, además de las variables x,p como series de Taylor.

In [111]:
#usamos los valores ya calculados en el orden 1 para x,p de primer orden, estos los incluyo en una lista junto 
#con  el polinomio que representa a x,p  tipo TaylorN 
P_θ=Taylor1([0.,vec1[1],θ],2)
P_p=Taylor1([0.,vec1[2],p],2)

 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( 1.0 p + 𝒪(‖x‖²)) t² + 𝒪(t³)

In [112]:
#aplicamos el mapeo
SO=EstandarMap(P_θ,P_p,ke)

2-element Array{Taylor1{TaylorN{Float64}},1}:
  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t² + 𝒪(t³)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( 0.3 θ + 1.3 p + 𝒪(‖x‖²)) t² + 𝒪(t³)

Para el lado derecho de la ecuación cohomológica necesitamos un vector de lambdas, pues recordemos  
$$ \Sigma_{n=1}^{\infty}a_{n}t^{n}\lambda^{n}$$
$$\Sigma_{n=1}^{\infty}b_{n}t^{n}\lambda^{n}$$


In [113]:
# como nos falta el otro lado de la ecuación cohomológica definimos un vector de lambdas
vλ=[0.0,λ,λ^2]

3-element Array{Float64,1}:
 0.0                
 0.5821091654199727 
 0.33885108046593715

In [116]:
#y definimos también un polinomio para las lambdas, del mismo orden que el polinomio de P_θ,P_p
# que juega la parte de a_{n}\lambda^{n}
θλt=Taylor1(vλ.*[0.,vec1[1],θ],2)
pλt=Taylor1(vλ.*[0.,vec1[2],p],2)
λvec=[θλt,pλt]

2-element Array{Taylor1{TaylorN{Float64}},1}:
   ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.33885108046593715 θ + 𝒪(‖x‖²)) t² + 𝒪(t³)
  ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( 0.33885108046593715 p + 𝒪(‖x‖²)) t² + 𝒪(t³)

Ahora ya tenemos las dos partes de la ecuación de invariancia,las igualamos para resolver. 

SO representa la parte de la primera composición mientras que $\lambda vec$ representa la parte derecha de la ecuación de invariancia. Al restarlas estamos resolviendo 
 $$f_{\epsilon}\circ P- P \circ \lambda=0 $$

En este caso no esta igualada a cero puesto que estamos tratando de calcular el siguente término, es decir van a diferir justo en ese término. 

In [117]:
#Restando ambos vectores que representan la ecuación cohomológica encontraremos los términos que faltan
# de empatar
Ecua=SO-λvec


2-element Array{Taylor1{TaylorN{Float64}},1}:
   ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( 0.6611489195340629 θ + 1.0 p + 𝒪(‖x‖²)) t² + 𝒪(t³)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( 0.3 θ + 0.9611489195340629 p + 𝒪(‖x‖²)) t² + 𝒪(t³)

En este paso nos podemos dar cuenta que sólo aprece el término de orden dos puesto que el de orden uno ya lo calculamos. 

Notemos que hay un pequeño error, pues aparece un número en el polinomio asociado a $\theta$ de orden 1, sin embargo este número es del orden de $10^{-16}$ lo cual nos dice que es un error numérico.

In [118]:

# de esta ecuación necesitamos solo los de segundo orden, así que los extraemos manualmente 
θ2=Ecua[1].coeffs[3]
p2=Ecua[2].coeffs[3]
vec2=[θ2,p2]

2-element Array{TaylorN{Float64},1}:
  0.6611489195340629 θ + 1.0 p + 𝒪(‖x‖²)
  0.3 θ + 0.9611489195340629 p + 𝒪(‖x‖²)

In [119]:
#calculamos ahora el jacobiano de este vector, ya que los coeficientes serán parte de la matriz que queremos
JSO=jacobian(vec2)

2×2 Transpose{Float64,Array{Float64,2}}:
 0.661149  1.0     
 0.3       0.961149

In [120]:
#calculamos su determinante para ver si es cero, 
det(JSO)

0.3354625696612776

In [121]:
# como es distinto de cero y la ecuación esta igualada a cero entonces la única solución es la trivial
θ2=0.
p2=0.
vec2=[0.,0.]


2-element Array{Float64,1}:
 0.0
 0.0

Esto concuerda con lo que habíamos encontrado antes, los coeficientes de orden par son cero. 

In [122]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(SO[1])
@show(typeof(SO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 𝒪(‖x‖²)) t² + 𝒪(t³)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
SO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t² + 𝒪(t³)
typeof(SO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.33885108046593715 θ + 𝒪(‖x‖²)) t² + 𝒪(t³)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( 0.6611489195340629 θ + 1.0 p + 𝒪(‖x‖²)) t² + 𝒪(t³)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                ORDEN 3

Para este orden comenzaremos a generar los vectores como en el caso anterior, lo que cambia en este caso es que
aparecen términos independientes en la ecuación comohológica. Esto resulta en que las solución ya no será la trivial como en el caso pasado.

In [123]:
#genramos un par de variables de tipo TaylorN de orden 3
θ,p = set_variables(Float64,"θ p",order=3)


2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖⁴)
  1.0 p + 𝒪(‖x‖⁴)

In [125]:
#generamos las variables con las que evaluamos el mapeo, notemos que deben de tener orden mayor al orden que estamos 
# calculando
P_θ=Taylor1([0.,vec1[1],θ2,θ],4)
P_p=Taylor1([0.,vec1[2],p2,p],4)


 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( 1.0 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)

In [126]:
#aplicamos el mapeo 
TO=EstandarMap(P_θ,P_p,ke)


2-element Array{Taylor1{TaylorN{Float64}},1}:
                       ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 1.0 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( 0.1358876056468051 + 0.3 θ + 1.3 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)

In [127]:
#creamos un vector de lambdas que formará parte del lado derecho de la ecuación cohomo.
vλ=[0,λ,λ^2, λ^3]
θλt=Taylor1([0.,vec1[1],θ2,θ].*vλ,4)
pλt=Taylor1([0.,vec1[2],p2,p].*vλ,4)



 ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( 0.19724831965168269 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)

In [128]:
#lo usaremos como vector pues el mapeo regresa un vector
λvec=[θλt,pλt]


2-element Array{Taylor1{TaylorN{Float64}},1}:
   ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.19724831965168269 θ + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
  ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( 0.19724831965168269 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)

In [129]:
#Escribimos la ecuación cohomo
Ecua=TO-λvec


2-element Array{Taylor1{TaylorN{Float64}},1}:
                        ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( 0.8027516803483173 θ + 1.0 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( 0.1358876056468051 + 0.3 θ + 1.1027516803483173 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)

In [130]:
#notamos de nuevo que sólo sobreviven los términos de orden 3 y que hay un pequeño error en el término 1 
θ3=Ecua[1].coeffs[4]
p3=Ecua[2].coeffs[4]

 0.1358876056468051 + 0.3 θ + 1.1027516803483173 p + 𝒪(‖x‖⁴)

In [131]:
#Dado que en este momento comienzan a aparecer términos independientes en x, p necesitamos una forma de extraerlos
# primero estraemos los coeficientes de x, p como antes con el jacobiano. 
JTO=jacobian([θ3,p3])

2×2 Transpose{Float64,Array{Float64,2}}:
 0.802752  1.0    
 0.3       1.10275

In [132]:
#calculamos el determinante 
det(JTO)

0.5852357644065422

In [133]:

#extraemos ahora los coeficientes independientes de x,p 
a= Ecua[1].coeffs[4].coeffs[1].coeffs[1]
b= Ecua[2].coeffs[4].coeffs[1].coeffs[1]
vecCoef=[-a,-b]


2-element Array{Float64,1}:
 -0.0               
 -0.1358876056468051

Dado que el determinante de la matriz es distinto de cero y los términos independientes son diferentes de cero entonces existe una solución no trivial. Para resolver este sitema utilizamos que :
$$JTO [\theta, p]^{T}=[a,b]^{T}$$
$$[\theta, p]^{T}=JTO^{-1}[a,b]^{T}$$

In [134]:

# entonces solo se trata de invertir el jacobiano y multiplicar con el vector del lado izquierdo
T3=JTO \ vecCoef


2-element Array{Float64,1}:
  0.2321929278956521 
 -0.18639326303323042

In [135]:
#los valores que obtuvimos son los nuevos coeficientes de orden 3
θ3=T3[1]
p3=T3[2]
v3=[θ3,p3]

2-element Array{Float64,1}:
  0.2321929278956521 
 -0.18639326303323042

In [136]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(TO[1])
@show(typeof(TO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
TO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 1.0 θ + 1.0 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
typeof(TO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.19724831965168269 θ + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( 0.8027516803483173 θ + 1.0 p + 𝒪(‖x‖⁴)) t³ + 𝒪(t⁵)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                        ORDEN 4

In [137]:
#creamos las variables de tipo TaylorN
θ,p = set_variables(Float64,"θ p",order=1)


2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖²)
  1.0 p + 𝒪(‖x‖²)

In [139]:
#creamos las variables del mapeo que ya toman en cuenta los coeficientes que calculamos
P_θ=Taylor1([0.,vec1[1],θ2,θ3,θ],5)
P_p=Taylor1([0.,vec1[2],p2,p3,p],5)


 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( - 0.18639326303323042 + 𝒪(‖x‖¹)) t³ + ( 1.0 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)

In [140]:
#Le aplicamos el mapeo estándar
CO=EstandarMap(P_θ,P_p,ke)


2-element Array{Taylor1{TaylorN{Float64}},1}:
                                             ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( - 0.03676575792769882 + 𝒪(‖x‖¹)) t³ + ( 0.3 θ + 1.3 p + 𝒪(‖x‖²)) t⁴ + ( - 0.026611022998050758 + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁶)

In [142]:
#creamos la parte derecha de la ecuación
vλ=[0., λ, λ^2, λ^3, λ^4]
θλt=Taylor1([0.,vec1[1],θ2,θ3,θ].*vλ,5)
pλt=Taylor1([0.,vec1[2],p2,p3,p].*vλ,5)
print(θλt,pλt)


 ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.11482005473293301 θ + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶) ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.11482005473293301 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)

In [143]:
#es más útil tener esta parte como un vector
λvec=[θλt,pλt]

2-element Array{Taylor1{TaylorN{Float64}},1}:
    ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.11482005473293301 θ + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
  ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.11482005473293301 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)

In [144]:
#escribimos la ecuación cohomo, para poder sacar el jacobiano
Ecua=CO-λvec


2-element Array{Taylor1{TaylorN{Float64}},1}:
                                            ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 0.885179945267067 θ + 1.0 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 0.3 θ + 1.185179945267067 p + 𝒪(‖x‖²)) t⁴ + ( - 0.026611022998050758 + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁶)

In [145]:
#obtenemos los valores de orden 4 
θ4=Ecua[1].coeffs[5]
p4=Ecua[2].coeffs[5]





 0.3 θ + 1.185179945267067 p + 𝒪(‖x‖²)

In [146]:
#calculamos el jacobiano de estos términos, para obtener solo los coeficientes
JCO=jacobian([θ4,p4])

2×2 Transpose{Float64,Array{Float64,2}}:
 0.88518  1.0    
 0.3      1.18518

In [147]:
#calculamos el determinante para saber si podemos resolver el sistema_
det(JCO)

0.7490975190831279

In [148]:
#dado que el determinante es distinto de cero y los valores independientes del sistema son cero
# la única solución es la trivial
T4=JCO\ [0.,0.]


2-element Array{Float64,1}:
 0.0
 0.0

In [149]:
#esta solución son los nuevos coeficientes del polinomio
θ4=T4[1]
p4=T4[2]

0.0

De nuevo dado que es un término de orden par, los coeficientes fueron cero.

In [150]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(CO[1])
@show(typeof(CO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 0.2321929278956521 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
CO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
typeof(CO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.11482005473293301 θ + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 0.885179945267067 θ + 1.0 p + 𝒪(‖x‖²)) t⁴ + 𝒪(t⁶)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                 QUINTO ORDEN

In [151]:
#creamos las variables que representan a x,p de tipo TaylorN
θ,p = set_variables(Float64,"θ p",order=1)


2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖²)
  1.0 p + 𝒪(‖x‖²)

In [152]:
#definimos nuestros nuevos polinomios tomando en cuenta todos los coeficientes ya calculados
P_θ=Taylor1([0.,vec1[1],θ2,θ3,θ4,θ],6)
P_p=Taylor1([0.,vec1[2],p2,p3,p4,p],6)


 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( - 0.18639326303323042 + 𝒪(‖x‖¹)) t³ + ( 1.0 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)

In [153]:
#aplicamos el mapeo
QO=EstandarMap(P_θ,P_p,ke)

2-element Array{Taylor1{TaylorN{Float64}},1}:
                             ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( - 0.03676575792769882 + 𝒪(‖x‖¹)) t³ + ( - 0.026611022998050758 + 0.3 θ + 1.3 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)

In [154]:
#creamos la otra parte de la ecuación cohomo
vλ=[0.,λ,λ^2,λ^3,λ^4,λ^5]
θλt=Taylor1([0.,vec1[1],θ2,θ3,θ4,θ].*vλ,6)
pλt=Taylor1([0.,vec1[2],p2,p3,p4,p].*vλ,6)
print(θλt,pλt)

 ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.06683780623406323 θ + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷) ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.06683780623406323 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)

In [155]:
#nos es más útil como vector
λvec=[θλt,pλt]

2-element Array{Taylor1{TaylorN{Float64}},1}:
    ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.06683780623406323 θ + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
  ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.06683780623406323 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)

In [156]:
#escribimos la ecuación cohomo
Ecua=QO-λvec

2-element Array{Taylor1{TaylorN{Float64}},1}:
                            ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 0.9331621937659368 θ + 1.0 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( - 0.026611022998050758 + 0.3 θ + 1.2331621937659367 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)

In [157]:
#obtenemos los coeficientes  que no son cero, es decir los de orden 5
θ5=Ecua[1].coeffs[6]
p5=Ecua[2].coeffs[6]


 - 0.026611022998050758 + 0.3 θ + 1.2331621937659367 p + 𝒪(‖x‖²)

In [158]:
#les calculamos el jacobiano
JQO=jacobian([θ5,p5])

2×2 Transpose{Float64,Array{Float64,2}}:
 0.933162  1.0    
 0.3       1.23316

In [159]:
#calculamos el determinante para ver si podemos encontrar una solución
det(JQO)

0.8507403380038367

In [160]:
#Dado que el determinante es distinto de cero buscamos los términos independientes
a= Ecua[1].coeffs[6].coeffs[1].coeffs[1]
b= Ecua[2].coeffs[6].coeffs[1].coeffs[1]
vecCoef=[-a,-b]

2-element Array{Float64,1}:
 -0.0                 
  0.026611022998050758

In [161]:
#como los térmimos independientes son diferentes de cero, y el determinante de la matriz también 
# podemos encontrar una solución 

T4=JQO \ vecCoef

2-element Array{Float64,1}:
 -0.03127984157950054 
  0.029189165588977693

In [162]:
#estos són los nuevos coeficientes de orden 5
θ5=T4[1]
p5=T4[2]


0.029189165588977693

In [163]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(QO[1])
@show(typeof(QO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 0.2321929278956521 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
QO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
typeof(QO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( 0.06683780623406323 θ + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 0.9331621937659368 θ + 1.0 p + 𝒪(‖x‖²)) t⁵ + 𝒪(t⁷)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                    Sexto orden

In [164]:
#creamos las variables que representan a x,p
θ,p = set_variables(Float64,"θ p",order=1)

2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖²)
  1.0 p + 𝒪(‖x‖²)

In [165]:
#creamos las variables que meteremos en el mapeo, las cuales son polinomios que ya tienen hasta el quinto orden calculado
P_θ=Taylor1([0.,vec1[1],θ2,θ3,θ4,θ5,θ],7)
P_p=Taylor1([0.,vec1[2],p2,p3,p4,p5,p],7)

 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( - 0.18639326303323042 + 𝒪(‖x‖¹)) t³ + ( 0.029189165588977693 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)

In [166]:
#les aplicamos el mapeo
SO=EstandarMap(P_θ,P_p,ke)

2-element Array{Taylor1{TaylorN{Float64}},1}:
                                           ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( - 0.002090675990522848 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( - 0.03676575792769882 + 𝒪(‖x‖¹)) t³ + ( 0.0019509397937700762 + 𝒪(‖x‖¹)) t⁵ + ( 0.3 θ + 1.3 p + 𝒪(‖x‖²)) t⁶ + ( 0.0038346622037622523 + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁸)

In [167]:
#creamos el otro lado de la ecuación cohomo
vλ=[0., λ, λ^2,λ^3, λ^4, λ^5, λ^6]
θλt=Taylor1([0.,vec1[1],θ2,θ3,θ4,θ5,θ].*vλ,7)
pλt=Taylor1([0.,vec1[2],p2,p3,p4,p5,p].*vλ,7)
print(θλt,pλt)

 ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.03890689960541239 θ + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸) ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.001950939793770077 + 𝒪(‖x‖¹)) t⁵ + ( 0.03890689960541239 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)

In [168]:
λvec=[θλt,pλt]

2-element Array{Taylor1{TaylorN{Float64}},1}:
  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.03890689960541239 θ + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
   ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.001950939793770077 + 𝒪(‖x‖¹)) t⁵ + ( 0.03890689960541239 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)

In [169]:
#escribimos la ecuación cohomo para obtener el sistema asociado a este orden
Ecua=SO-λvec

2-element Array{Taylor1{TaylorN{Float64}},1}:
                                             ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 3.469446951953614e-18 + 𝒪(‖x‖¹)) t⁵ + ( 0.9610931003945876 θ + 1.0 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( - 8.673617379884035e-19 + 𝒪(‖x‖¹)) t⁵ + ( 0.3 θ + 1.2610931003945876 p + 𝒪(‖x‖²)) t⁶ + ( 0.0038346622037622523 + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁸)

In [170]:
#extraemos solo los coeficientes que nos interesa calcular, en principio los demás términos deben ser cero pues ya los 
# calculamos previamente
θ6=Ecua[1].coeffs[7]
p6=Ecua[2].coeffs[7]

 0.3 θ + 1.2610931003945876 p + 𝒪(‖x‖²)

In [171]:
#calculamos el jacobiano para obetener el sistema
JSO=jacobian([θ6,p6])

2×2 Transpose{Float64,Array{Float64,2}}:
 0.961093  1.0    
 0.3       1.26109

In [172]:
#calculamos el determinante del jacobiano para saber si hay solución
det(JSO)

0.912027877744457

In [173]:
#dado que el determinante del jacobiano es distinto de cero y los términos independientes son iguales a cero 
# la solución a este sistema es la trivial
T6=JSO\ [0.,0.]


2-element Array{Float64,1}:
 0.0
 0.0

In [174]:
#por tanto los coeficientes de orden 6 son cero
θ6=T6[1]
p6=T6[2]

0.0

In [175]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(SO[1])
@show(typeof(SO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 0.2321929278956521 + 𝒪(‖x‖¹)) t³ + ( - 0.03127984157950054 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
SO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( - 0.002090675990522848 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
typeof(SO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.03890689960541239 θ + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 3.469446951953614e-18 + 𝒪(‖x‖¹)) t⁵ + ( 0.9610931003945876 θ + 1.0 p + 𝒪(‖x‖²)) t⁶ + 𝒪(t⁸)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}

                                                Séptimo orden

In [176]:
#creamos las variables de x,p
θ,p = set_variables(Float64,"θ p",order=1)

2-element Array{TaylorN{Float64},1}:
  1.0 θ + 𝒪(‖x‖²)
  1.0 p + 𝒪(‖x‖²)

In [177]:
#calculamos los polinomios asociados a la variedad que ya tienen todos los términos previos
P_θ=Taylor1([0.,vec1[1],θ2,θ3,θ4,θ5,θ6,θ],8)
P_p=Taylor1([0.,vec1[2],p2,p3,p4,p5,p6,p],8)


 ( - 0.5831757547123116 + 𝒪(‖x‖¹)) t + ( - 0.18639326303323042 + 𝒪(‖x‖¹)) t³ + ( 0.029189165588977693 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)

In [178]:
#les aplicamos el mapeo
SEO=EstandarMap(P_θ,P_p,ke)

2-element Array{Taylor1{TaylorN{Float64}},1}:
                           ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( - 0.002090675990522848 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
  ( - 1.0018322839695704 + 𝒪(‖x‖¹)) t + ( - 0.03676575792769882 + 𝒪(‖x‖¹)) t³ + ( 0.0019509397937700762 + 𝒪(‖x‖¹)) t⁵ + ( 0.0038346622037622523 + 0.3 θ + 1.3 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)

In [179]:
#calculamos el otro lado de la ecuación cohomo
vλ=[0., λ, λ^2,λ^3, λ^4, λ^5, λ^6,λ^7]
θλt=Taylor1([0,vec1[1],θ2,θ3,θ4,θ5,θ6,θ].*vλ,8)
pλt=Taylor1([0,vec1[2],p2,p3,p4,p5,p6,p].*vλ,8)
print(θλt,pλt)

 ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.022648062858385273 θ + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹) ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.001950939793770077 + 𝒪(‖x‖¹)) t⁵ + ( 0.022648062858385273 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)

In [180]:
λvec=[θλt,pλt]

2-element Array{Taylor1{TaylorN{Float64}},1}:
  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.022648062858385273 θ + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
   ( - 0.33947195186874646 + 𝒪(‖x‖¹)) t + ( - 0.0367657579276988 + 𝒪(‖x‖¹)) t³ + ( 0.001950939793770077 + 𝒪(‖x‖¹)) t⁵ + ( 0.022648062858385273 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)

In [181]:
#escribimos la ecua. cohomo
Ecua=SEO-λvec

2-element Array{Taylor1{TaylorN{Float64}},1}:
                             ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 3.469446951953614e-18 + 𝒪(‖x‖¹)) t⁵ + ( 0.9773519371416147 θ + 1.0 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
  ( - 0.6623603321008239 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( - 8.673617379884035e-19 + 𝒪(‖x‖¹)) t⁵ + ( 0.0038346622037622523 + 0.3 θ + 1.2773519371416149 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)

In [182]:
#dado que los términos anteriores ya los calculamos solo extraemos los términos de orden 7 
θ7=Ecua[1].coeffs[8]
p7=Ecua[2].coeffs[8]

 0.0038346622037622523 + 0.3 θ + 1.2773519371416149 p + 𝒪(‖x‖²)

In [183]:
#los ponemos como un vector y le calculamos el jacobianos para obtener el sistema
JSEO=jacobian([θ7,p7])

2×2 Transpose{Float64,Array{Float64,2}}:
 0.977352  1.0    
 0.3       1.27735

In [184]:
#calculamos el determinante del sistema para saber si hay solución
det(JSEO)

0.9484223901769514

In [185]:
#extraemos los términos independienets de la ecuación cohomo
a= Ecua[1].coeffs[8].coeffs[1].coeffs[1]
b= Ecua[2].coeffs[8].coeffs[1].coeffs[1]
vecCoef=[-a,-b]

2-element Array{Float64,1}:
 -0.0                  
 -0.0038346622037622523

In [186]:
#dado que el determinante de la matriz es distinto de cero y los términos independientes también, podemos encontrar 
# una solución distinta de la trivial
T7=JSEO\vecCoef

2-element Array{Float64,1}:
  0.004043200839076355
 -0.003951630172323878

In [187]:
#estos nuevos elementos son los coeficientes de orden 7 en el polinomio
θ7=T7[1]
p7=T7[2]

-0.003951630172323878

In [188]:
#queremos revisar la estabilidad de las variables tx,x
@show(P_θ)
@show(typeof(P_θ))
@show(SEO[1])
@show(typeof(SEO[1]))
@show(θλt)
@show(typeof(θλt))
@show(Ecua[1])
@show(typeof(Ecua[1]))

P_θ =  ( - 0.8123460094785507 + 𝒪(‖x‖¹)) t + ( 0.2321929278956521 + 𝒪(‖x‖¹)) t³ + ( - 0.03127984157950054 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
typeof(P_θ) = Taylor1{TaylorN{Float64}}
SEO[1] =  ( - 1.3955217641908624 + 𝒪(‖x‖¹)) t + ( 0.04579966486242168 + 𝒪(‖x‖¹)) t³ + ( - 0.002090675990522848 + 𝒪(‖x‖¹)) t⁵ + ( 1.0 θ + 1.0 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
typeof(SEO[1]) = Taylor1{TaylorN{Float64}}
θλt =  ( - 0.4728740576098044 + 𝒪(‖x‖¹)) t + ( 0.04579966486242169 + 𝒪(‖x‖¹)) t³ + ( - 0.0020906759905228514 + 𝒪(‖x‖¹)) t⁵ + ( 0.022648062858385273 θ + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
typeof(θλt) = Taylor1{TaylorN{Float64}}
Ecua[1] =  ( - 0.922647706581058 + 𝒪(‖x‖¹)) t + ( - 1.3877787807814457e-17 + 𝒪(‖x‖¹)) t³ + ( 3.469446951953614e-18 + 𝒪(‖x‖¹)) t⁵ + ( 0.9773519371416147 θ + 1.0 p + 𝒪(‖x‖²)) t⁷ + 𝒪(t⁹)
typeof(Ecua[1]) = Taylor1{TaylorN{Float64}}


Taylor1{TaylorN{Float64}}